In [2]:
import torch
import torch.utils.data as data
import torchvision.transforms as transforms
import numpy as np
from pytorch_nsynth_lib.guanaset import Gset
from IPython.display import Audio

import librosa
import librosa.display
import phase_operation
import spectrograms_helper as spec_helper
# from tqdm import tqdm
import h5py
import matplotlib.pyplot as plt

In [ ]:
# import spec_ops as spec_ops
# import phase_operation as phase_op

In [2]:
train_data = h5py.File('../data/GSet_melspec_IF_pitch.hdf5', 'w')

In [3]:
toFloat = transforms.Lambda(lambda x: x / np.iinfo(np.int16).max)
dataset = Gset(
        "../data",
        transform=toFloat)
loader = data.DataLoader(dataset, batch_size=1, shuffle=True)

In [ ]:
# for samples_left, samples_right, ptch in loader:
#     print(samples_left, ptch)

In [3]:
def expand(mat):
    expand_vec = np.expand_dims(mat[:,34],axis=1)
    expanded = np.hstack((mat,expand_vec))
    return expanded

In [4]:
def reduce(mat):
    return np.delete(mat, np.s_[32:], axis=1)

In [5]:
spec_list=[]
pitch_list=[]
IF_list =[]
mel_spec_list=[]
mel_IF_list=[]

pitch_set =set()
count=0
for samples_left, samples_right, pitches in loader:

    sample_left = samples_left.data.numpy().squeeze()
    sample_right = samples_right.data.numpy().squeeze()
    
    sample_mid = ( sample_left + sample_right ) / 2
    sample_side = ( sample_left - sample_right ) / 2

    """재정의, left가 mid고 right가 side임"""
    sample_left = sample_mid
    sample_right = sample_side
    
    spec_left = librosa.stft(sample_left, n_fft=2048, hop_length = 512)
    spec_right = librosa.stft(sample_right, n_fft=2048, hop_length = 512)
    
#     print(spec_left[100])
    
    """ log10 말고 log로 처리"""
    magnitude_left = np.log(np.abs(spec_left)+ 1.0e-6)[:1024]
    magnitude_right = np.log(np.abs(spec_right)+ 1.0e-6)[:1024]
    
    angle_left = np.angle(spec_left)
    angle_right = np.angle(spec_right)

    IF_left = phase_operation.instantaneous_frequency(angle_left,time_axis=1)[:1024]
    IF_right = phase_operation.instantaneous_frequency(angle_right,time_axis=1)[:1024]
    
    magnitude_left = reduce(magnitude_left)
    magnitude_right = reduce(magnitude_right)   
    
    IF_left = reduce(IF_left)
    IF_right = reduce(IF_right)
    
    logmelmag2_left, mel_p_left = spec_helper.specgrams_to_melspecgrams(magnitude_left, IF_left)
    logmelmag2_right, mel_p_right = spec_helper.specgrams_to_melspecgrams(magnitude_right, IF_right)
    
#     print(magnitude_left.shape)
    
    assert magnitude_left.shape ==(1024, 32)
    assert magnitude_right.shape ==(1024, 32)
    assert IF_left.shape ==(1024, 32)
    assert IF_right.shape ==(1024, 32)
    
    pitch_list.append(pitches)
    mel_spec_list.append([logmelmag2_left,logmelmag2_right])
    mel_IF_list.append([mel_p_left,mel_p_right])
    pitch_set.add(pitches)
    
    count+=1
    if count%1000==0:
        print(count)
    if count%4000==0:
        np.save('../np_array_temp/' + 'pitch_list_' + str(count), pitch_list)
        np.save('../np_array_temp/' + 'mel_spec_list_' + str(count), mel_spec_list)        
        np.save('../np_array_temp/' + 'mel_IF_list_' + str(count), mel_IF_list)        
        pitch_list=[]
        mel_spec_list=[]
        mel_IF_list=[]
        print(str(count) + " saved")
np.save('../np_array_temp/' + 'pitch_list_' + str(count), pitch_list)
np.save('../np_array_temp/' + 'mel_spec_list_' + str(count), mel_spec_list)        
np.save('../np_array_temp/' + 'mel_IF_list_' + str(count), mel_IF_list)
print(str(count) + "saved")

1000
2000
3000
4000
4000 saved
5000
6000
7000
8000
8000 saved
9000
10000
11000
12000
12000 saved
13000
14000
15000
16000
16000 saved
17000
18000
19000
20000
20000 saved
21000
22000
23000
24000
24000 saved
25000
25904saved


In [10]:
# train_data.create_dataset("Spec", data=spec_list)
# train_data.create_dataset("IF", data=IF_list)
train_data.create_dataset("pitch", data=pitch_list)
train_data.create_dataset("mel_Spec", data=mel_spec_list)
train_data.create_dataset("mel_IF", data=mel_IF_list)


MemoryError: Unable to allocate 14.2 GiB for an array with shape (25904, 2, 1024, 36) and data type float64

In [8]:
for k in train_data.keys():
    dataset = train_data[k]
    print (dataset)
    print (dataset.dtype)
    print (dataset[()])

<HDF5 dataset "mel_IF": shape (10, 2, 1024, 36), type "<f8">
float64
[[[[ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ...  0.00000000e+00
     0.00000000e+00  0.00000000e+00]
   [ 7.66124149e-17  2.75555820e-01  3.32721830e-01 ... -2.81660408e-03
     2.98100929e-02  0.00000000e+00]
   [-6.42737734e-01 -2.54223571e-02 -3.41613834e-01 ... -5.51196512e-03
     1.28736772e-01  0.00000000e+00]
   ...
   [ 5.12659649e+01 -2.35854910e-01  1.25154509e-01 ...  6.59267527e-02
     4.64143174e-01  0.00000000e+00]
   [ 4.75776679e+01 -3.15740719e-03 -2.42915778e-02 ... -2.83082248e-01
    -4.44370069e-01  0.00000000e+00]
   [ 4.52023915e+01 -7.53955288e-01 -3.35944730e-01 ... -3.17348681e-01
    -8.55995339e-01  0.00000000e+00]]

  [[ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ...  0.00000000e+00
     0.00000000e+00  0.00000000e+00]
   [ 6.43241608e-17  2.75558623e-01  3.32725887e-01 ... -4.86724297e-03
     2.99782748e-02  0.00000000e+00]
   [ 6.42737734e-01  6.89062111e-01 -3.41494260e-0